In [4]:
"""
在labeled_data.csv中加’block_id‘和’touch_type‘列,生成labeled_tactile_dataset.csv
遇到DT,ST,P,G文件时，将所有未标记的数据标记为：
‘block_id’ = -1  ’touch_type‘= ’NC‘
"""
import os
import pandas as pd

global_block_id = 0

def process_labeledData(df, tactile_type):
    """处理单个labeled_data.csv文件，标记数据块和背景数据"""
    """Process a single labeled_data.csv file to tag data blocks and background data."""
    global global_block_id
    labeled_blocks = []
    total_blocks = 0
    
    in_block = False
    block_start = 0
    block_info = []

    # if tactile_type == 'NC':
    #     # Mark touched data as 'ST_abandon' and untouched data as 'NC'
    #     df.loc[df['label'] == 1, ['block_id', 'touch_type']] = [-1, 'ST_abandon']
    #     df.loc[df['label'] == 0, ['block_id', 'touch_type']] = [-2, 'NC']
    #     return df, total_blocks, block_info

    for i in range(len(df)):
        if df.iloc[i, df.columns.get_loc('label')] == 1 and not in_block:
            in_block = True
            block_start = i
        elif df.iloc[i, df.columns.get_loc('label')] == 0 and in_block:
            in_block = False
            block = df.iloc[block_start:i].copy()
            block['touch_type'] = tactile_type
            block['block_id'] = global_block_id  # 添加全局唯一数据块标识符 # Add global unique block identifier
            labeled_blocks.append(block)
            block_info.append((global_block_id, block_start, i - 1))  # 记录block信息 # Record block info
            global_block_id += 1  # 增加全局唯一数据块标识符 # Increment global block ID
            total_blocks += 1

    if in_block:
        block = df.iloc[block_start:].copy()
        block['touch_type'] = tactile_type
        block['block_id'] = global_block_id  # 添加全局唯一数据块标识符
        labeled_blocks.append(block)
        block_info.append((global_block_id, block_start, len(df) - 1))  # 记录block信息
        global_block_id += 1  # 增加全局唯一数据块标识符
        total_blocks += 1
    
    
    df['block_id'] = -1
    df['touch_type'] = 'NC'
    
    # 将处理后的数据块合并回原始DataFrame
    # Merge processed blocks back into the original DataFrame
    for block in labeled_blocks:
        df.update(block)
    
    return df, total_blocks, block_info

def calculate_zero_counts(df, block_info):
    """计算相邻数据块之间0的个数"""
    """Calculate the count of zeros between adjacent data blocks."""
    zero_counts = []
    for i in range(len(block_info) - 1):
        end_of_current_block = block_info[i][2]
        start_of_next_block = block_info[i + 1][1]
        zero_count = (df.iloc[end_of_current_block + 1:start_of_next_block]['label'] == 0).sum()
        zero_counts.append((block_info[i][0], block_info[i + 1][0], zero_count))
    return zero_counts

def determine_double_taps(zero_counts):
    """根据0的个数决定哪些单拍应该合并为双拍"""
    """Decide which single taps should merge into double taps based on zero counts."""
    zero_counts_values = [count[2] for count in zero_counts]
    zero_counts_values.sort()
    n = len(zero_counts_values)
    max_value_index = int((n / 2) / (n + 1) * n)
    max_value = zero_counts_values[max_value_index]
    min_value = zero_counts_values[0]
    
    double_tap_blocks = []
    for start_block_id, end_block_id, zero_count in zero_counts:
        if min_value <= zero_count <= max_value:
            double_tap_blocks.append((start_block_id, end_block_id))
    return double_tap_blocks

def merge_continuous_double_taps(double_tap_blocks):
    """合并连续的双拍块对"""
    """Merge continuous double tap block pairs."""
    merged_double_tap_blocks = []
    current_start = None
    current_end = None
    
    for start_block_id, end_block_id in double_tap_blocks:
        if current_start is None:
            current_start = start_block_id
            current_end = end_block_id
        elif start_block_id == current_end:
            current_end = end_block_id
        else:
            merged_double_tap_blocks.append((current_start, current_end))
            current_start = start_block_id
            current_end = end_block_id
    
    if current_start is not None:
        merged_double_tap_blocks.append((current_start, current_end))
    
    return merged_double_tap_blocks

def merge_double_taps(df, block_info, double_tap_blocks, start_block_id):
    """合并确定的双拍数据块"""
    """Merge identified double tap data blocks."""
    merged_double_tap_blocks = merge_continuous_double_taps(double_tap_blocks)
    merged_blocks = []
    new_block_id = start_block_id

    for start_block_id, end_block_id in merged_double_tap_blocks:
        start_row = next(bi[1] for bi in block_info if bi[0] == start_block_id)
        end_row = next(bi[2] for bi in block_info if bi[0] == end_block_id)
        
        merged_block = df.iloc[start_row:end_row + 1].copy()
        merged_block['block_id'] = new_block_id  # 使用新的连续编号
        merged_block['touch_type'] = 'DT'  # 标记为双拍
        
        merged_blocks.append(merged_block)
        new_block_id += 1

    return merged_blocks, new_block_id


def process_double_tap_folder(df, tactile_type, start_block_id):
    """处理双拍数据，将挨得非常近的两个连续块标记为双拍类型"""
    """Process double tap data and label closely adjacent blocks as double tap type."""
    df, total_blocks, block_info = process_labeledData(df, tactile_type)
    zero_counts = calculate_zero_counts(df, block_info)
    double_tap_blocks = determine_double_taps(zero_counts)
    merged_blocks, new_block_id = merge_double_taps(df, block_info, double_tap_blocks, start_block_id)
    
    # 将双拍块更新回df
    for block in merged_blocks:
        df.update(block)
    
    return df, total_blocks

def process_single_folder(data_folder, folder_name, start_block_id):
# def process_single_folder(data_folder, folder_name, start_block_id,output_file= None):
    """处理指定文件夹下的labeled_data.csv，并打印块和行的信息"""
    """Process labeled_data.csv in the specified folder and print block and row info."""
    folder_path = os.path.join(data_folder, folder_name)
    
    if os.path.isdir(folder_path):  # Check if it's a directory
        file_path = os.path.join(folder_path, 'labeled_data.csv')
        
        if os.path.isfile(file_path):
            tactile_type = folder_name.split('-')[1][1:]
            df = pd.read_csv(file_path)
            
            if tactile_type == 'DT':
                df, total_blocks = process_double_tap_folder(df, tactile_type, start_block_id)
            elif tactile_type in ['ST', 'G', 'P']:
                df, total_blocks, _ = process_labeledData(df, tactile_type)
            elif tactile_type == 'NC':
                df, total_blocks, _ = process_labeledData(df, tactile_type)
            else:
                print(f"未知的触碰类型: {tactile_type}")
                return pd.DataFrame(), 0
            
            print(f"文件夹 {folder_name}: 数据长度 = {len(df)}, 数据块个数 = {total_blocks}")
            # output_path = os.path.join(output_file, '2_specific_labeled_tactile_dataset.csv')
            # df.to_csv(output_path, index=False)
            return df, total_blocks
        else:
            print(f"文件夹 {folder_name} 下没有找到labeled_data.csv文件")
            return pd.DataFrame(), 0
    else:
        print(f"{folder_name} 不是有效的文件夹")
        return pd.DataFrame(), 0

def process_all_folders(data_folder, output_file):
    """处理所有子文件夹，合并所有数据块，生成labeled_tactile_dataset文件"""
    """Process all subfolders, merge all data blocks, and create labeled_tactile_dataset."""
    all_data = []
    total_blocks_count = 0
    
    for folder_name in os.listdir(data_folder):
        df, total_blocks = process_single_folder(data_folder, folder_name, global_block_id)
        if not df.empty:
            all_data.append(df)
            total_blocks_count += total_blocks
    
    if all_data:
        result_df = pd.concat(all_data, ignore_index=True)
        output_path = os.path.join(output_file, '2_labeled_tactile_dataset_post123.csv')
        
        # 删除已存在的文件
        if os.path.exists(output_path):
            os.remove(output_path)
        
        # 保存新文件
        result_df.to_csv(output_path, index=False)
        print(f"2_labeled_tactile_dataset.csv 文件已生成，total length = {len(result_df)}, total_block = {total_blocks_count}")
    else:
        print("没有数据块被处理")

# 设置数据文件夹路径
data_folder = '../DATA/Labeled_data/ModelLabel/'
folder_name = "0213_2LCNN_Post1_ST"
output_file = '../DATA/'
process_all_folders(data_folder, output_file)
# process_single_folder(data_folder, folder_name, start_block_id=1,output_file = output_file)

IndexError: list index out of range

In [8]:
# 验证生成的数据 
import os
import pandas as pd
df = pd.read_csv("../DATA/2_labeled_tactile_dataset.csv")
df.head(100)
# dt.iloc[240:300]
# dt[dt['touch_type']=='DT']

,index,time,tau_J0,tau_J1,tau_J2,tau_J3,tau_J4,tau_J5,tau_J6,tau_J_d0,...,etau_J0,etau_J1,etau_J2,etau_J3,etau_J4,etau_J5,etau_J6,label,block_id,touch_type
0,1,0.000000,-0.339380,-47.427479,1.917226,18.934702,0.336326,2.153210,-0.018444,0.0,...,0.339380,47.427479,-1.917226,-18.934702,-0.336326,-2.153210,0.018444,0,-1,NC
1,2,0.004982,-0.339380,-47.427479,1.917226,18.934702,0.336326,2.153210,-0.018444,0.0,...,0.339380,47.427479,-1.917226,-18.934702,-0.336326,-2.153210,0.018444,0,-1,NC
2,3,0.009981,-0.339380,-47.427479,1.917226,18.934702,0.336326,2.153210,-0.018444,0.0,...,0.339380,47.427479,-1.917226,-18.934702,-0.336326,-2.153210,0.018444,0,-1,NC
3,4,0.014978,-0.339380,-47.427479,1.917226,18.934702,0.336326,2.153210,-0.018444,0.0,...,0.339380,47.427479,-1.917226,-18.934702,-0.336326,-2.153210,0.018444,0,-1,NC
4,5,0.019982,-0.299707,-47.467152,1.917226,18.934702,0.336326,2.164196,-0.038585,0.0,...,0.299707,47.467152,-1.917226,-18.934702,-0.336326,-2.164196,0.038585,0,-1,NC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0.474983,-0.375085,-47.427479,1.881520,18.895029,0.336326,2.174267,0.021839,0.0,...,0.375085,47.427479,-1.881520,-18.895029,-0.336326,-2.174267,-0.021839,0,-1,NC
96,97,0.479985,-0.299707,-47.352100,1.917226,18.895029,0.327171,2.174267,0.001698,0.0,...,0.299707,47.352100,-1.917226,-18.895029,-0.327171,-2.174267,-0.001698,0,-1,NC
97,98,0.484982,-0.299707,-47.352100,1.917226,18.895029,0.327171,2.174267,0.001698,0.0,...,0.299707,47.352100,-1.917226,-18.895029,-0.327171,-2.174267,-0.001698,0,-1,NC
98,99,0.489980,-0.339380,-47.352100,1.952932,18.895029,0.327171,2.143139,0.001698,0.0,...,0.339380,47.352100,-1.952932,-18.895029,-0.327171,-2.143139,-0.001698,0,-1,NC


In [9]:
print(df['block_id'].nunique())
print("---")
print(df.groupby('touch_type')['block_id'].nunique())
print("---")
print(df['block_id'].unique())


901
---
touch_type
G     150
NC      1
P     150
ST    600
Name: block_id, dtype: int64
---
[ -1   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222

In [10]:
df[df['touch_type']=='NC']
# nc['block_id'].unique()
# nc[nc['block_id']==0]
df[df['block_id']==-1]

,index,time,tau_J0,tau_J1,tau_J2,tau_J3,tau_J4,tau_J5,tau_J6,tau_J_d0,...,etau_J0,etau_J1,etau_J2,etau_J3,etau_J4,etau_J5,etau_J6,label,block_id,touch_type
0,1,0.000000,-0.339380,-47.427479,1.917226,18.934702,0.336326,2.153210,-0.018444,0.0,...,0.339380,47.427479,-1.917226,-18.934702,-0.336326,-2.153210,0.018444,0,-1,NC
1,2,0.004982,-0.339380,-47.427479,1.917226,18.934702,0.336326,2.153210,-0.018444,0.0,...,0.339380,47.427479,-1.917226,-18.934702,-0.336326,-2.153210,0.018444,0,-1,NC
2,3,0.009981,-0.339380,-47.427479,1.917226,18.934702,0.336326,2.153210,-0.018444,0.0,...,0.339380,47.427479,-1.917226,-18.934702,-0.336326,-2.153210,0.018444,0,-1,NC
3,4,0.014978,-0.339380,-47.427479,1.917226,18.934702,0.336326,2.153210,-0.018444,0.0,...,0.339380,47.427479,-1.917226,-18.934702,-0.336326,-2.153210,0.018444,0,-1,NC
4,5,0.019982,-0.299707,-47.467152,1.917226,18.934702,0.336326,2.164196,-0.038585,0.0,...,0.299707,47.467152,-1.917226,-18.934702,-0.336326,-2.164196,0.038585,0,-1,NC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406169,4366,21.824992,-0.113244,-45.785023,0.199391,18.371347,0.502036,2.362865,-0.069713,0.0,...,0.113244,45.785023,-0.199391,-18.371347,-0.502036,-2.362865,0.069713,0,-1,NC
406170,4367,21.830011,-0.113244,-45.824696,0.235097,18.411020,0.502036,2.373852,0.021839,0.0,...,0.113244,45.824696,-0.235097,-18.411020,-0.502036,-2.373852,-0.021839,0,-1,NC
406171,4368,21.834998,-0.113244,-45.824696,0.235097,18.411020,0.502036,2.373852,0.021839,0.0,...,0.113244,45.824696,-0.235097,-18.411020,-0.502036,-2.373852,-0.021839,0,-1,NC
406172,4369,21.839996,-0.148950,-45.860401,0.163686,18.335642,0.483726,2.341808,-0.038585,0.0,...,0.148950,45.860401,-0.163686,-18.335642,-0.483726,-2.341808,0.038585,0,-1,NC
